In [226]:
import pandas as pd
# Import required libraries and dependencies

import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error



# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [227]:
attributes_df = pd.read_csv("../Resources/songAttributes_1999-2019.csv")


billboard_df = pd.read_csv("../Resources/billboardHot100_1999-2019.csv")


artists_df = pd.read_csv("../Resources/artistDf.csv")


weekly_df = pd.read_csv("../Resources/spotifyWeeklyTop200Streams.csv")

weekly_df

,Unnamed: 0,Name,Artist,Features,Streams,Week
0,0,In My Feelings,Drake,NaN,30747676,2018-07-20
1,1,Lucid Dreams,Juice WRLD,NaN,12930705,2018-07-20
2,2,Nonstop,Drake,NaN,12312859,2018-07-20
3,3,God is a woman,Ariana Grande,NaN,10771324,2018-07-20
4,4,SAD!,XXXTENTACION,NaN,10503061,2018-07-20
...,...,...,...,...,...,...
26390,26390,R.I.P. SCREW,Travis Scott,NaN,1521806,2018-09-14
26391,26391,Rich & Sad,Post Malone,NaN,1518345,2018-09-14
26392,26392,The Way Life Goes,Lil Uzi Vert,Oh Wonder,1516966,2018-09-14
26393,26393,broken,lovelytheband,NaN,1486767,2018-09-14


In [228]:
weekly_df

,Unnamed: 0,Name,Artist,Features,Streams,Week
0,0,In My Feelings,Drake,NaN,30747676,2018-07-20
1,1,Lucid Dreams,Juice WRLD,NaN,12930705,2018-07-20
2,2,Nonstop,Drake,NaN,12312859,2018-07-20
3,3,God is a woman,Ariana Grande,NaN,10771324,2018-07-20
4,4,SAD!,XXXTENTACION,NaN,10503061,2018-07-20
...,...,...,...,...,...,...
26390,26390,R.I.P. SCREW,Travis Scott,NaN,1521806,2018-09-14
26391,26391,Rich & Sad,Post Malone,NaN,1518345,2018-09-14
26392,26392,The Way Life Goes,Lil Uzi Vert,Oh Wonder,1516966,2018-09-14
26393,26393,broken,lovelytheband,NaN,1486767,2018-09-14


In [229]:
weekly_df.drop(columns=["Unnamed: 0", "Name"], inplace=True)

In [230]:
grouped = weekly_df.groupby("Artist")["Streams"].sum()
grouped_reset = grouped.reset_index()
grouped_dict = grouped_reset.to_dict(orient="split")
# total_streams_df = pd.DataFrame(grouped_dict)


# Create a DataFrame from the dictionary
total_streams_df = pd.DataFrame(grouped_dict['data'], columns=grouped_dict['columns'])

# Display the resulting DataFrame
total_streams_df

,Artist,Streams
0,*NSYNC,5690510
1,2 Chainz,275432430
2,21 Savage,914888503
3,22 Savage,4982496
4,5 Seconds of Summer,236805582
...,...,...
590,"benny blanco, Jesse, Swae Lee, Jesse, Swae Lee",1808602
591,"benny blanco, Juice WRLD feat. Brendon Urie, J...",50309105
592,blackbear,152112104
593,gnash,18720111


In [231]:
weekly_df = weekly_df.drop_duplicates(subset = "Artist", keep = "first").reset_index()

In [232]:
weekly_df.drop(columns=["index"], inplace=True)

In [233]:
weekly_df

,Artist,Features,Streams,Week
0,Drake,NaN,30747676,2018-07-20
1,Juice WRLD,NaN,12930705,2018-07-20
2,Ariana Grande,NaN,10771324,2018-07-20
3,XXXTENTACION,NaN,10503061,2018-07-20
4,Tyga,Offset,9777479,2018-07-20
...,...,...,...,...
590,The xx,NaN,1153955,2017-01-20
591,WizKid,NaN,1242932,2017-04-07
592,Tame Impala,NaN,1633712,2019-04-19
593,Mitchell Tenpenny,NaN,1462815,2019-01-11


In [234]:
final_df = weekly_df.merge(total_streams_df, how="inner", on="Artist")
final_df

,Artist,Features,Streams_x,Week,Streams_y
0,Drake,NaN,30747676,2018-07-20,3441947366
1,Juice WRLD,NaN,12930705,2018-07-20,1445959305
2,Ariana Grande,NaN,10771324,2018-07-20,1598012598
3,XXXTENTACION,NaN,10503061,2018-07-20,2872157309
4,Tyga,Offset,9777479,2018-07-20,313734165
...,...,...,...,...,...
590,The xx,NaN,1153955,2017-01-20,2220061
591,WizKid,NaN,1242932,2017-04-07,1242932
592,Tame Impala,NaN,1633712,2019-04-19,4011775
593,Mitchell Tenpenny,NaN,1462815,2019-01-11,1462815


In [235]:
final_df = final_df.drop(columns=["Streams_x"])
final_df

,Artist,Features,Week,Streams_y
0,Drake,NaN,2018-07-20,3441947366
1,Juice WRLD,NaN,2018-07-20,1445959305
2,Ariana Grande,NaN,2018-07-20,1598012598
3,XXXTENTACION,NaN,2018-07-20,2872157309
4,Tyga,Offset,2018-07-20,313734165
...,...,...,...,...
590,The xx,NaN,2017-01-20,2220061
591,WizKid,NaN,2017-04-07,1242932
592,Tame Impala,NaN,2019-04-19,4011775
593,Mitchell Tenpenny,NaN,2019-01-11,1462815


In [236]:
final_df.rename(columns={"Streams_y":"Total streams"}, inplace=True)

In [237]:
artists_df["Artist"] = artists_df["Artist"].str.lower()
final_df["Artist"] = final_df["Artist"].str.lower()

In [238]:
artist_week = artists_df.merge(final_df, how="outer", on="Artist")

In [239]:
artist_week["Total streams"].notna().value_counts()

False    905
True     595
Name: Total streams, dtype: int64

In [245]:
artist_week = artist_week.dropna(subset = ["Total streams"])

In [249]:
artist_week.drop(columns=["X"], inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3548\2268797100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artist_week.drop(columns=["X"], inplace=True)


In [257]:
artist_week.reset_index(inplace=True)
artist_week.drop(columns=["index"])

,Artist,Followers,Genres,NumAlbums,YearFirstAlbum,Gender,Group.Solo,Features,Week,Total streams
0,ed sheeran,52698756.0,"pop,uk pop",8.0,2011.0,M,Solo,NaN,2018-07-20,9.361553e+08
1,justin bieber,30711450.0,"canadian pop,dance pop,pop,post-teen pop",10.0,2009.0,M,Solo,NaN,2018-12-14,4.543449e+07
2,jonas brothers,3069527.0,"boy band,dance pop,pop,post-teen pop",10.0,2006.0,M,Group,NaN,2019-04-12,1.825584e+08
3,drake,41420478.0,"canadian hip hop,canadian pop,hip hop,pop rap,...",11.0,2010.0,M,Solo,NaN,2018-07-20,3.441947e+09
4,chris brown,9676862.0,"dance pop,pop,pop rap,r&b,rap",6.0,2005.0,M,Solo,NaN,2017-11-10,1.383342e+08
...,...,...,...,...,...,...,...,...,...,...
590,the xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-20,2.220061e+06
591,wizkid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-04-07,1.242932e+06
592,tame impala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-19,4.011775e+06
593,mitchell tenpenny,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-11,1.462815e+06


In [258]:
artist_week.to_csv("../Resources/ArtistWeek.csv")